In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

def animate_CVRP_solution(routes, node_coords, save=False, filename='cvrp_animation.mp4'):
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Plot the nodes
    for i, (x, y) in enumerate(node_coords):
        ax.scatter(x, y, c='blue' if i == 0 else 'red', zorder=2)
        ax.text(x, y, f'{i}', fontsize=9, ha='right')
    
    # Prepare the routes with depot at start and end
    vehicle_routes = []
    vehicle_cumulative_distances = []
    total_route_lengths = []
    for route in routes:
        full_route = [0] + route + [0]
        vehicle_routes.append(full_route)
        
        # Compute cumulative distances
        distances = [0.0]
        for i in range(1, len(full_route)):
            node_a = full_route[i - 1]
            node_b = full_route[i]
            coord_a = node_coords[node_a]
            coord_b = node_coords[node_b]
            dist = np.hypot(coord_b[0] - coord_a[0], coord_b[1] - coord_a[1])
            distances.append(distances[-1] + dist)
        vehicle_cumulative_distances.append(distances)
        total_route_lengths.append(distances[-1])
    
    # Colors for vehicles
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
    
    # Prepare data for animation
    lines = []
    markers = []
    for vehicle, route in enumerate(vehicle_routes):
        color = colors[vehicle % len(colors)]
        line, = ax.plot([], [], c=color, label=f'Vehicle {vehicle + 1}', lw=2)
        lines.append(line)
        # Add a marker to represent the vehicle
        marker, = ax.plot([], [], marker='o', c=color, markersize=8)
        markers.append(marker)
    
    # Set plot limits
    x_coords, y_coords = zip(*node_coords)
    margin = 10
    ax.set_xlim(min(x_coords)-margin, max(x_coords)+margin)
    ax.set_ylim(min(y_coords)-margin, max(y_coords)+margin)
    
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('Vehicle Routing Problem Solution')
    #ax.legend()
    ax.grid(True)
    
    # Total number of frames in the animation
    total_frames = 200
    
    # Function to update the animation at each frame
    def update(frame):
        t = frame / total_frames  # Normalized time from 0 to 1
        for i, line in enumerate(lines):
            route = vehicle_routes[i]
            cumulative_distances = vehicle_cumulative_distances[i]
            total_length = total_route_lengths[i]
            elapsed_distance = t * total_length
            
            # Find current segment
            idx = np.searchsorted(cumulative_distances, elapsed_distance, side='right') - 1
            if idx >= len(route) - 1:
                idx = len(route) - 2
                fraction = 1.0
            else:
                # Fraction along the segment
                fraction = ((elapsed_distance - cumulative_distances[idx]) /
                            (cumulative_distances[idx+1] - cumulative_distances[idx]))
            
            node_a = route[idx]
            node_b = route[idx+1]
            coord_a = node_coords[node_a]
            coord_b = node_coords[node_b]
            x = coord_a[0] + fraction * (coord_b[0] - coord_a[0])
            y = coord_a[1] + fraction * (coord_b[1] - coord_a[1])
            
            # Update line data
            route_nodes = route[:idx+1]
            x_coords_line = [node_coords[node][0] for node in route_nodes]
            y_coords_line = [node_coords[node][1] for node in route_nodes]
            x_coords_line.append(x)
            y_coords_line.append(y)
            line.set_data(x_coords_line, y_coords_line)
            
            # Update marker position
            markers[i].set_data([x], [y])
        return lines + markers
    
    # Create the animation
    ani = animation.FuncAnimation(fig, update, frames=total_frames+1, interval=50, blit=True, repeat=False)
    
    # Save or display the animation
    if save:
        # Determine writer based on file extension
        ext = filename.split('.')[-1]
        if ext.lower() == 'mp4':
            # Try to use ffmpeg writer
            try:
                Writer = animation.writers['ffmpeg']
                writer = Writer(fps=20, metadata=dict(artist='Me'), bitrate=1800)
            except KeyError:
                raise RuntimeError("ffmpeg writer is not available on your system. Please install ffmpeg or choose a different file format.")
        elif ext.lower() == 'gif':
            # Use PillowWriter or ImageMagick
            try:
                from matplotlib.animation import PillowWriter
                writer = PillowWriter(fps=20)
            except ImportError:
                raise RuntimeError("PillowWriter is not available. Please install Pillow or choose a different file format.")
        else:
            raise ValueError(f"Unsupported file extension: .{ext}. Please use .mp4 or .gif.")
        
        ani.save(filename, writer=writer)
    else:
        plt.show()


In [ ]:
# Require: brew install ffmpeg
animate_CVRP_solution(best_routes, node_coords, save=True, filename='cvrp_animation.mp4')

In [ ]:
animate_CVRP_solution(best_routes, node_coords, save=True, filename='cvrp_animation.gif')